# Assignment #2: Language models
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment

## Submission

Once you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [399]:
import bz2
import math
import os
import regex as re
import requests
import sys
import numpy
from zipfile import ZipFile

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [527]:
# You may have to adjust the path
corpus = open('./Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [401]:
pattern = 'Nils Holgersson'
width = 25

In [528]:
# spaces match tabs and newlines
pattern = re.sub(' ', r'\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub(r'\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
#for match in re.finditer(concordance, clean_corpus):
    #print(match.group(1))
# print the string with 0..width characters on either side

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [529]:
def tokenize(text):
    words = re.findall(r'\p{L}+', text)
    return words

In [530]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [406]:
original = tokenize(corpus)
unique = []
for word in original:
    if word not in unique:
        unique.append(word)
print(len(unique))

44256


Lowercased text

In [407]:
original = tokenize(corpus)
unique = []
lowercase = [x.lower() for x in original]
for word in lowercase:
    if word not in unique:
        unique.append(word)
print(len(unique))

41032


### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [524]:
nonletter = "[^.;:?!\p{L}]+"

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [531]:
def clean(paragraph):
    cleanParagraph = re.sub(nonletter, " ", paragraph)
    cleanerParagraph = cleanParagraph.replace("  ", " ")
    return cleanerParagraph

In [410]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [532]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga som hette Back Kajsa. Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.'

#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces.

In [533]:
sentence_boundaries = r"[.;:?!]+\s+(\p{Lu})"

##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [534]:
sentence_markup = r" </s>\n<s> \1"

##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [414]:
text = re.sub(sentence_boundaries, sentence_markup, test_para)

In [415]:
print(text)

En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.


Insert markup codes in the beginning and end of the text

In [416]:
text = "<s> " + text + " </s>"

In [417]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden. </s>


Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex.

In [418]:
text = re.sub("\s+", " ", text)
text = re.sub("\.", "", text)

In [419]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s> <s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [535]:
def segment_sentences(text):
    text = re.sub(sentence_boundaries, sentence_markup, text)
    text = re.sub("\.", "", text)
    text = "<s> " + text + " </s>"
    text = re.sub("\s+", " ", text)
    text = text.lower()
    return text

In [513]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s> <s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Estimate roughly the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus

In [536]:
corpus = segment_sentences(clean(corpus))

The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this.

In [537]:
print(corpus[-557:])

<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s> <s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s> <s> prästens ord tycktes redan ha gått i uppfyllelse </s> <s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s> <s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>


You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [538]:
words = corpus.split()

The five last lines of the corpus should like this:

In [539]:
print(words[-101:])

['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']


### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [540]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [541]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 59047),
 ('selma', 51),
 ('lagerlöf', 269),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 316),
 ('genom', 688),
 ('sverige', 54),
 ('</s>', 59047),
 ('första', 525),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 77),
 ('den', 11611),
 ('kristliga', 2),
 ('dagvisan', 2)]

#### Bigrams

In [542]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [543]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('<s>', 'selma'), 8),
 (('selma', 'lagerlöf'), 11),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 17),
 (('</s>', '<s>'), 59046),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', 'stockholm'), 10),
 (('stockholm', '</s>'), 24),
 (('<s>', 'den'), 1375),
 (('den', 'kristliga'), 2)]

In [591]:
test_bigrams = bigrams(words)
test_bigrams[('<s>', 'selma')]

8

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus.

### Computing the likelihood of a sentence

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_.

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [571]:
def unigram_lm(freq, words):
    lenCorpus = len(corpus.split())
    Cwi = []
    for word in words:
        Cwi.append(unigrams(corpus.split())[word])
    print("======================================================")
    print("wi      C(wi)         #words          P(wi)")
    print("======================================================")
    for i in range(0, len(Cwi)):
        print(words[i] + "\t" + str(Cwi[i]) + "\t\t" + str(lenCorpus) + "\t\t" + str(Cwi[i]/lenCorpus))
    print("======================================================")
    probList = [Cwi[i]/lenCorpus for i in range(0, len(words))]
    mul = numpy.prod(probList)
    print(mul)
    pk = [prob * math.log(prob) for prob in probList]
    S = -sum(pk)
    print("Prob. unigrams:   " + str(mul))
    print("Geometric mean prob.: " + str(pow(mul, 1/len(words))))
    print("Entropy rate:    " + str(S))
    perplexity = pow(1/mul, 1/len(words))
    print("Perplexity:      " + str(perplexity))
    return perplexity
    

In [553]:
sentence = 'det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>']

In [570]:
perplexity_unigrams = unigram_lm(frequency, sent_words)

wi      C(wi)         #words          P(wi)
det	21076		1041564		0.020234954357101435
var	12089		1041564		0.011606583944913611
en	13505		1041564		0.012966077936641436
gång	1328		1041564		0.0012750056645582989
en	13505		1041564		0.012966077936641436
katt	16		1041564		1.536151403082288e-05
som	16288		1041564		0.01563802128337769
hette	97		1041564		9.31291788118637e-05
nils	87		1041564		8.35282325425994e-05
</s>	59047		1041564		0.05669070743612491
5.333137479311845e-27
Prob. unigrams:   5.333137479311845e-27
Geometric mean prob.: 0.0023588388822626993
Entropy rate:    0.2090612424307462
Perplexity:      423.93739034891485


In [572]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

423

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [744]:
def bigram_lm(dic, bigDic, text):
    prevWord="<s>"
    unigramPredict = 1 #dic[prevWord]/sum(dic.values())
    entropyRate = 0
    bigrams = list(frequency_bigrams.keys())
    print('=====================================================')
    print('wi','\t' ,'wi+1', '\t', 'Ci,i+1',' ', 'C(i)', '\t','P(wi+1|wi)')
    print('=====================================================')
   
    for index in range(len(text)):
        word = text[index]
        #prob = dic[word]/sum(dic.values())
        ci1 = bigDic.get((prevWord,word))
        if bigDic.get((prevWord,word)) == None:
            ci1 = 0
            totalProb = dic.get(word)/(sum(dic.values()))
        else:   
            totalProb = ci1/dic.get(prevWord)
  
        unigramPredict *= totalProb
        entropyRate += math.log(totalProb,2)
        print(prevWord, '\t',word , '\t',ci1, '\t', dic.get(prevWord), '\t', totalProb ) 
        prevWord = word
    print('=====================================================')
    print('Prob. unigrams:',unigramPredict)
    print('Geometric mean prob.:', unigramPredict ** (1./len(text)))
    print('Entropy rate:', -entropyRate/len(text))
    print('Perplexity: ', 2 ** ( -entropyRate/len(text)))
    return  math.floor(2 ** ( -entropyRate/len(text)))

In [745]:
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)

wi 	 wi+1 	 Ci,i+1   C(i) 	 P(wi+1|wi)
<s> 	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21076 	 0.18215031315240082
var 	 en 	 712 	 12089 	 0.05889651749524361
en 	 gång 	 702 	 13505 	 0.05198074787115883
gång 	 en 	 20 	 1328 	 0.015060240963855422
en 	 katt 	 6 	 13505 	 0.00044427989633469083
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 8.35282325425994e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
Prob. unigrams: 2.3767736839150797e-19
Geometric mean prob.: 0.013727731714283164
Entropy rate: 6.186762926627613
Perplexity:  72.84524645535865


In [697]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

48

In addition to this sentence, _Det var en gång en katt som hette Nils_, write three other sentences that will form your test set and run your programs on them. You will insert them in your report.

In [797]:
test_set = 'Han stod i predikstolen och väntade, medan sista versen av predikstolspsalmen sjöngs. Prästen skulle ha velat störta ner på sina knän och be dem om förbarmande. Medan han läste upp inledningen, sköljde blodvåg efter blodvåg upp i hans ansikte. Det var vreden. Han hade läst upp inledningen, och han böjde sig ner för att läsa Fader vår.'
test_sentences = segment_sentences(clean(test_set)).split()
sent = ""
sentenceArr = []
for word in test_sentences:
    if word == '</s>':
        sent += word
        sentenceArr.append(sent)
        sent = ""
    elif word != '<s>':
        sent += word + " "
for sentence in sentenceArr:
    sentence = sentence.split()
    print("\nUnigrams: ")
    unigram_lm(frequency, sentence)
    print("\nBigrams:")
    bigram_lm(frequency, frequency_bigrams, sentence)


Unigrams: 
wi      C(wi)         #words          P(wi)
han	21580		1041564		0.020718842049072355
stod	1192		1041564		0.0011444327952963044
i	16508		1041564		0.015849242101301505
predikstolen	22		1041564		2.1122081792381457e-05
och	36356		1041564		0.03490520025653728
väntade	223		1041564		0.00021410110180459385
medan	568		1041564		0.0005453337480942122
sista	276		1041564		0.00026498611703169465
versen	4		1041564		3.84037850770572e-06
av	5433		1041564		0.005216194108091293
predikstolspsalmen	1		1041564		9.6009462692643e-07
sjöngs	3		1041564		2.8802838807792895e-06
</s>	59047		1041564		0.05669070743612491
2.692058607844137e-44
Prob. unigrams:   2.692058607844137e-44
Geometric mean prob.: 0.0004451107049951685
Entropy rate:    0.46941361756050537
Perplexity:      2246.6321047274173

Bigrams:
wi 	 wi+1 	 Ci,i+1   C(i) 	 P(wi+1|wi)
<s> 	 han 	 5058 	 59047 	 0.08566057547377513
han 	 stod 	 132 	 21580 	 0.0061167747914735865
stod 	 i 	 56 	 1192 	 0.04697986577181208
i 	 predikstolen 	 5 	 

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program this function following the model of your bigram counting function.

In [700]:
def trigrams(words):
    trigrams = [tuple(words[inx:inx + 3]) for inx in range(len(words) - 2)]
    frequencies = {}
    for trigram in trigrams:
        if trigram in frequencies:
            frequencies[trigram] += 1
        else:
            frequencies[trigram] = 1
    return frequencies

In [701]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [702]:
starting_text = 'De'.lower()
starting_text

'de'

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [703]:
cand_nbr = 5

In [704]:
candidates = {}
for key in frequency_bigrams.keys():
    if key[0] == '<s>' and key[1].startswith(starting_text):
        candidates[key] = frequency_bigrams[key]
        
sortedCandidates = dict(sorted(candidates.items(), key=lambda x: x[0][1].lower()))
p = sorted(sortedCandidates, key=sortedCandidates.get, reverse=True)[:cand_nbr]
current_word_predictions_1 = [i[1] for i in p]

In [705]:
current_word_predictions_1

['det', 'de', 'den', 'detta', 'denna']

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [706]:
current_text = "Det var en ".lower()
current_text

'det var en '

Tokenize this text and return a list of tokens. Call it `tokens`.

In [707]:
tokens = tokenize(current_text)

In [708]:
tokens

['det', 'var', 'en']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [709]:
candidates = {}
for key in frequency_trigrams.keys():
    if key[0] == 'var' and key[1] == 'en':
        candidates[key] = frequency_trigrams[key]
        
sortedCandidates = dict(sorted(candidates.items(), key=lambda x: x[0][2].lower()))
p = sorted(sortedCandidates, key=sortedCandidates.get, reverse=True)[:cand_nbr]
next_word_predictions = [i[2] for i in p]

In [710]:
next_word_predictions

['stor', 'liten', 'gammal', 'god', 'sådan']

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [711]:
current_text = "Det var en g".lower()

In [712]:
candidates = {}
for key in frequency_trigrams.keys():
    if key[0] == 'var' and key[1] == 'en' and key[2].startswith('g'):
        candidates[key] = frequency_trigrams[key]
        
sortedCandidates = dict(sorted(candidates.items(), key=lambda x: x[0][2].lower()))
p = sorted(sortedCandidates, key=sortedCandidates.get, reverse=True)[:cand_nbr]
current_word_predictions_2 = [i[2] for i in p]
print(current_word_predictions_2)

['gammal', 'god', 'gång', 'ganska', 'glädje']


In [713]:
current_word_predictions_2

['gammal', 'god', 'gång', 'ganska', 'glädje']

## Checked answers

The automatic marking system will check these answers: `(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)`

In [714]:
(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)

(423,
 48,
 ['det', 'de', 'den', 'detta', 'denna'],
 ['stor', 'liten', 'gammal', 'god', 'sådan'],
 ['gammal', 'god', 'gång', 'ganska', 'glädje'])

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [715]:
STIL_ID = ["ha6882os-s"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "2-language_models.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the perplexities and predictions.

In [747]:
import json
ANSWER = json.dumps({'perplexity_unigrams': perplexity_unigrams,
                     'perplexity_bigrams': perplexity_bigrams,
                     'current_word_predictions_1': current_word_predictions_1,
                     'next_word_predictions': next_word_predictions,
                     'current_word_predictions_2': current_word_predictions_2})
ANSWER

'{"perplexity_unigrams": 423, "perplexity_bigrams": 72, "current_word_predictions_1": ["det", "de", "den", "detta", "denna"], "next_word_predictions": ["stor", "liten", "gammal", "god", "s\\u00e5dan"], "current_word_predictions_2": ["gammal", "god", "g\\u00e5ng", "ganska", "gl\\u00e4dje"]}'

The checking script will accept perplexities with a margin, like for instance:
```
(421, 72, ['det', 'de', 'den', 'detta', 'denna'], ['stor', 'liten', 'gammal', 'god', 'sådan'], ['gammal', 'god', 'gång', 'ganska', 'glädje'])
```

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [748]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [749]:
ASSIGNMENT = 2
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [750]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'correct',
 'signature': 'ed18ad2de12ee002efb60c2154aec19a8a2a7fcd8b86090e96bb982d8f7bd4cd08dd35d7ea8c845e00eb5362c951028230d10fdaa06d23b957525fddb513ad77',
 'submission_id': '4bdbd646-880f-4536-b2c8-5a1f15613baf'}

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. You will include the regular expression you use to segment the text and the unigram and bigram tables for three sentences and _Det var en gång en katt som hette Nils_.
2. Execute the Jupyter notebook by Peter Norvig here: <a href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">https://nbviewer.jupyter.org/url/norvig.com/ipython/How to Do Things with Words.ipynb</a>. Just run all the cells and be sure that you understand the code. You will find the data here: <a href="http://norvig.com/ngrams/">http://norvig.com/ngrams/</a>.
3. In your report, after the description of your program, you will describe one experiment with Norvig's notebook and a long string of words your will create yourself or copy from a text you like. You will remove all the punctuation and white spaces from this string. You will set this string in lowercase letters. You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. You will comment the segmentation results you obtain with the unigram and bigram models.

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 23, 2022. You will have only three submission attempts. The deadline for the second and third ones are one week after you are noticed of your result.